# MOFs LM Probing for Crystal System

In [ ]:
from baseline import PromptSet

from tqdm.auto import tqdm
from nltk.corpus import stopwords

import pandas as pd

from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline, logging

logging.set_verbosity_error()  # avoid irritating transformers warnings

## Read in the Triples

In [ ]:
file = "../data/mof_crystalSystem.csv"

In [ ]:
df = pd.read_csv(file)
df.head()

In [ ]:
df.shape

In [ ]:
df.ObjectEntity.nunique()

## Probing BERT

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-cased")
model = AutoModelForMaskedLM.from_pretrained("bert-large-cased")

In [ ]:
mask_token = tokenizer.mask_token
mask_token

## Creat the Prompt Pipeline

In [ ]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Create Prompts

In [ ]:
def create_crystalSystem_prompt1(sub, mask):
    return f"{sub} is an MOF. {sub} has the crystal system {mask}."
    
def create_crystalSystem_prompt2(sub, mask):
    return f"{sub} is an MOF. {sub} has an crystal family {mask}."

def create_crystalSystem_prompt3(sub, mask):
    return f"{sub} is an MOF. {sub} has an crystal system code {mask}."

def create_crystalSystem_prompt4(sub, mask):
    return f"{sub} is an MOF. The crystal system of {sub} is {mask}."
    
def create_crystalSystem_prompt5(sub, mask):
    return f"{sub} is an crystal. {sub} is in the family {mask}."
    
def create_crystalSystem_prompt6(sub, mask):
    return f"{sub} is an metal organic framework. {sub}'s crystal system is {mask}." 
    
def create_crystalSystem_prompt7(sub, mask):
    return f"{sub} is an MOF. {sub} is an type of {mask} crystal system."
    
def create_crystalSystem_prompt8(sub, mask):
    return f"{sub} is an MOF. {sub} is in the class of {mask} crystal family."
    
def create_crystalSystem_prompt9(sub, mask):
    return f"{sub} has SBUs and organic linkers. The crystal system of {sub} is {mask}."
    
def create_crystalSystem_prompt10(sub, mask):
    return f"{sub} is an MOF. The {mask} describes {sub}'s crystal structure."
    
def create_crystalSystem_prompt11(sub, mask):
    return f"{sub} is a MOF. The crystal family  of {sub} is {mask}."
    
def create_crystalSystem_prompt12(sub, mask):
    return f"As a MOF, {sub} has a crystal system {mask}."

In [ ]:
processors = [create_crystalSystem_prompt1, create_crystalSystem_prompt2,
              create_crystalSystem_prompt3, create_crystalSystem_prompt4,
              create_crystalSystem_prompt5, create_crystalSystem_prompt6,
              create_crystalSystem_prompt7, create_crystalSystem_prompt8,
              create_crystalSystem_prompt9, create_crystalSystem_prompt10,
              create_crystalSystem_prompt11, create_crystalSystem_prompt12
             ]

## Check Whether the Predicted Tokens Match

In [ ]:
# Compute a column indicating whether the predictedToken in the ObjectEntity list
def hasPredicted(row):
    token = row['predictedToken']
    objectEntities = row['ObjectEntity']
    #print(objectEntities)
    if pd.notna(objectEntities):
        types = objectEntities.split(",") # get individual types
        for tp in types: # for each type
            type_words = tp.lower().split() # get the individual words in the type
            if token.lower() in type_words: # check if a type word was predicted
                return 1
       
    return 0

In [ ]:
# test what the function does
def hasPredicted_demo(objectEntities, token):
    #print(objectEntities)
    if pd.notna(objectEntities):
        types = objectEntities.split(",") # get individual types
        for tp in types: # for each type
            type_words = tp.lower().split() # get the individual words in the type
            print(type_words)
            if token.lower() in type_words: # check if a type word was predicted
                return 1
       
    return 0

In [ ]:
hasPredicted_demo('rhombohedral', 'Framework')

## Test Small

In [ ]:
processors = [create_crystalSystem_prompt1, create_crystalSystem_prompt2, create_crystalSystem_prompt3]

In [ ]:
processors[0]('MOF-1', '[MASK]')

In [ ]:
mofs_sub =df[:10]

In [ ]:
prompts = PromptSet([processors[0](row.SubjectEntity, mask_token) for _, row in mofs_sub.iterrows()])

In [ ]:
prompts.__getitem__(9)

In [ ]:
for i in prompts:
    print(i)

In [ ]:
# Run the model
outputs = []
for out in tqdm(pipe(prompts, batch_size=8), total=len(prompts)):
    outputs.append(out)

In [ ]:
outputs

In [ ]:
results = []
for idx, row in mofs_sub.iterrows():
    for output in outputs[idx]:
        record = {}
        record['SubjectEntity'] = row['SubjectEntity']
        record['Relation'] = row['Relation']
        record['ObjectEntity'] = row['ObjectEntity']
        record['prompt']= prompts[idx]
        record['predictedScore'] = output['score']
        record['predictedToken'] = output['token_str']
        results.append(record)

In [ ]:
predicted_df = pd.DataFrame(results)
predicted_df

### Evaluate on Small

In [ ]:
predicted_df.loc[10]['ObjectEntity'].split(",")

In [ ]:
predicted_df['predicted'] = predicted_df.apply(hasPredicted, axis=1)

In [ ]:
predicted_df.head()

In [ ]:
predicted_df['predicted'].sum()

In [ ]:
predicted_df[predicted_df.predicted == 1]

In [ ]:
# return the top-k rows based on predictedScore
def top(df, col, n=10):
    return df.sort_values(by=col, ascending=False)[:n]

In [ ]:
topk_df = predicted_df.groupby('SubjectEntity', as_index=False).apply(top, col='predictedScore', n=100)

In [ ]:
# check whether the predicted is 1 for each group
def predictedOne(df, col):
    total = df[col].sum()
    if total > 0:
        return 1
    else:
        return 0

In [ ]:
topk_agg_df = topk_df.groupby('SubjectEntity', as_index=False).apply(predictedOne, col='predicted')

In [ ]:
topk_agg_df.columns = ['SubjectEntity', 'predicted']
topk_agg_df

In [ ]:
topk_agg_df.predicted.mean()

## Test BERT

In [ ]:
def prompt_probe(model, tokenizer, prompt_processor, df_sub, mask_token, \
                 subjectCol, relationCol, objectCol, top_k=100):

    pipe = pipeline(
        task="fill-mask",
        model=model,
        tokenizer=tokenizer,
        top_k=top_k,
    )

    prompts = PromptSet([prompt_processor(row.SubjectEntity, mask_token) for _, row in df_sub.iterrows()])

    outputs = []
    for out in tqdm(pipe(prompts, batch_size=8), total=len(prompts)):
        outputs.append(out)

    results = []
    for idx, row in df_sub.iterrows():
        for output in outputs[idx]:
            record = {}
            record['SubjectEntity'] = row[subjectCol]
            record['Relation'] = row[relationCol]
            record['ObjectEntity'] = row[objectCol]
            record['prompt']= prompts[idx]
            record['predictedScore'] = output['score']
            record['predictedToken'] = output['token_str']
            results.append(record)

    return pd.DataFrame(results)

In [ ]:
# return the top-k rows based on predictedScore
def top(df, col, n=10):
    return df.sort_values(by=col, ascending=False)[:n]

In [ ]:
# check whether the predicted is 1 for each group
def predictedOne(df, col):
    total = df[col].sum()
    if total > 0:
        return 1
    else:
        return 0

In [ ]:
# Assume the predicted_df has a binary column 'predicted' indicating whether ObjectEntity has been predicted
# 1 for yes, 0 for no
def evaluate_acc_n(predicted_df, relation, prompt_template, at_k=5):
    topk_df = predicted_df.groupby('SubjectEntity', as_index=False).apply(top, col='predictedScore', n=at_k)
    topk_agg_df = topk_df.groupby('SubjectEntity', as_index=False).apply(predictedOne, col='predicted')
    topk_agg_df.columns = ['SubjectEntity', 'predicted']
    result = {}
    result['Relation'] = relation
    result['prompt_template'] = prompt_template
    result['acc'] = topk_agg_df.predicted.mean()
    result['at_k'] = at_k

    return result

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntity', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasCrystalSystem', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

In [ ]:
hasType_df = pd.DataFrame(acc_list)

In [ ]:
hasType_df.sample(10)

In [ ]:
result_df.to_csv("../data/predicted_hasCrystalSystem_BERT.csv", index=None)

In [ ]:
hasType_df.to_csv("../data/predicted_hasCrystalSystem_BERT_eval.csv", index=None)

In [ ]:
hasType_df[hasType_df['at_k'] == 1].sort_values('acc', ascending=False)

In [ ]:
hasType_df.sort_values('acc', ascending=False)

## Probing MatBERT

In [ ]:
# MatBERT
tokenizer = AutoTokenizer.from_pretrained("../model/model/matbert-base-cased")
model = AutoModelForMaskedLM.from_pretrained("../model/model/matbert-base-cased")

In [ ]:
mask_token = tokenizer.mask_token
mask_token

## Creat the Prompt Pipeline

In [ ]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Run

In [ ]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntity', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasCrystalSystem', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

In [ ]:
hasType_df = pd.DataFrame(acc_list)

In [ ]:
hasType_df.sample(10)

In [ ]:
result_df.to_csv("../data/predicted_hasCrystalSystem_MatBERT.csv", index=None)

In [ ]:
hasType_df.to_csv("../data/predicted_hasCrystalSystem_MatBERT_eval.csv", index=None)

In [ ]:
hasType_df[hasType_df['at_k'] == 1].sort_values('acc', ascending=False)

In [ ]:
hasType_df.sort_values('acc', ascending=False)

## Probing MatSciBERT

In [ ]:
# MatSciBERT
tokenizer = AutoTokenizer.from_pretrained('m3rg-iitd/matscibert')
model = AutoModelForMaskedLM.from_pretrained('m3rg-iitd/matscibert')

In [ ]:
mask_token = tokenizer.mask_token
mask_token

## Creat the Prompt Pipeline

In [ ]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Run

In [ ]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntity', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasCrystalSystem', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

In [ ]:
hasType_df = pd.DataFrame(acc_list)

In [ ]:
hasType_df.sample(10)

In [ ]:
result_df.to_csv("../data/predicted_hasCrystalSystem_MatSciBERT.csv", index=None)

In [ ]:
hasType_df.to_csv("../data/predicted_hasCrystalSystem_MatSciBERT_eval.csv", index=None)

In [ ]:
hasType_df[hasType_df['at_k'] == 1].sort_values('acc', ascending=False)

In [ ]:
hasType_df.sort_values('acc', ascending=False)

## Probing SciBERT

In [ ]:
# SciBERT
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_cased')
model = AutoModelForMaskedLM.from_pretrained('allenai/scibert_scivocab_cased')

In [ ]:
mask_token = tokenizer.mask_token
mask_token

## Creat the Prompt Pipeline

In [ ]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Run

In [ ]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntity', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasCrystalSystem', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

In [ ]:
hasType_df = pd.DataFrame(acc_list)

In [ ]:
hasType_df.sample(10)

In [ ]:
result_df.to_csv("../data/predicted_hasCrystalSystem_SciBERT.csv", index=None)

In [ ]:
hasType_df.to_csv("../data/predicted_hasCrystalSystem_SciBERT_eval.csv", index=None)

In [ ]:
hasType_df[hasType_df['at_k'] == 1].sort_values('acc', ascending=False)

In [ ]:
hasType_df.sort_values('acc', ascending=False)

## Probing RoBERTa

In [ ]:
# RoBERTa
tokenizer = AutoTokenizer.from_pretrained("roberta-large")
model = AutoModelForMaskedLM.from_pretrained("roberta-large")

In [ ]:
mask_token = tokenizer.mask_token
mask_token

## Creat the Prompt Pipeline

In [ ]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Run

In [ ]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntity', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasCrystalSystem', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

In [ ]:
hasType_df = pd.DataFrame(acc_list)

In [ ]:
hasType_df.sample(10)

In [ ]:
result_df.to_csv("../data/predicted_hasCrystalSystem_RoBERTa.csv", index=None)

In [ ]:
hasType_df.to_csv("../data/predicted_hasCrystalSystem_RoBERTa_eval.csv", index=None)

In [ ]:
hasType_df[hasType_df['at_k'] == 1].sort_values('acc', ascending=False)

In [ ]:
hasType_df.sort_values('acc', ascending=False)